# SQL 4 Data Science Project

# Database Cleaning and Data Analytics with SQL

## Bhejane Online Trading Store

## BY </b> IPINSANMI OMOLAYO 


© Explore Data Science Academy

<a id='Context'></a>
### 1. Context

The Bhejane trading store is an online retailer specialising in Covid essential items. The store has recently been struggling with the management of its database-related inventory system. We've been hired as a consultant to fix the problem. 

 
**Mission:**

A denormalised database consisting of two tables has been provided. The task is to transform the database such that is in third normal form (3NF). To ensure that a consistent normalisation process is followed by each student, An Entity Relationship Diagram (ERD) has been attached to guide on what tables need to be produced.

![Bhejane company logo](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Bhejane.png)

<div align="center" >
    Bhejane, or the Black Rhino. Image by <a href="https://commons.wikimedia.org/wiki/File:Black_Rhino_(Diceros_bicornis)_browsing_..._(46584052962).jpg">Wikimedia Commons</a>
</div>

<a id='Imports'></a>
### 2. Imports


In [5]:
#Install sql magic

!pip install sql_magic

In [6]:
# import needed packages
import sqlite3
import csv
from sqlalchemy import create_engine
%load_ext sql_magic

# Load SQLite database
engine  = create_engine("sqlite:///bhejane.db")
%config SQL.conn_name ='engine'

The sql_magic extension is already loaded. To reload it, use:
  %reload_ext sql_magic


<a id='Data_description'></a>
### 3. Data description

The original database consist of 2 tables. 
* Product Table
* Transaction Table

The `Product` table consists of the stock of all inventory that Bhejane has on hand currently, or has had on hand historically. Items which are in this table are able to be purchased, and a record of all sales (transactions) in 2020 is notorised in the `Transactions` table. To link the tables - the `barcode` can be used. Any item in the `Transactions` table, must therefore appear in the `Products` table. 

In [7]:
# Read the data
import pandas as pd
data_description = pd.read_excel('Data Description.xlsx')
data_description

,Table Name,Column Name,Desciption
0,Products,Width,Width of the product once assembled
1,Products,Length,Length of the product once assembled
2,Products,Height,Height of the product once assembled
3,Products,Barcode,The unique product identifier
4,Products,Quantity,Number of goods in stock
5,Products,Brand,Product brand name relating to product company
6,Products,NavigationPath,Navigation path to specific product
7,Products,Colour,Name default colour for the product
8,Products,StockCountry,Country where the stock was bought from
9,Products,ProductDescription,Descriptive product name


<a id='Setting_up'></a>
### 4. Setting up the database

In [8]:
#setting up the database
conn = sqlite3.connect('bhejane.db')
cursor = conn.cursor()

#### Creating both Products and Transaction Tables

In [9]:
%%read_sql
DROP TABLE IF EXISTS "Products";
DROP TABLE IF EXISTS "Transactions";

CREATE TABLE "Products" (
    "Width"   REAL,
    "Length"  REAL,
    "Height"  REAL,
    "Barcode" VARCHAR(150),
    "Quantity" REAL,
    "Brand" VARCHAR(150), 
    "NavigationPath" VARCHAR(150),
    "Colour" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "ProductDescription" VARCHAR(150),
    "PackType" VARCHAR(150), 
    "Volume_litre" REAL, 
    "Warranty" VARCHAR(150), 
    "Weight_kg" REAL,
    "ItemDescription" VARCHAR(150), 
    "Price" REAL
);


CREATE TABLE "Transactions" (
    "CartID" INTEGER,
    "Barcode" VARCHAR(150), 
    "Total" REAL,
    "UserName" VARCHAR(150), 
    "InvoiceDate" DATETIME
);

Query started at 12:21:04 AM W. Central Africa Standard Time; Query executed in 0.00 m

#### Load Data into Product and Transaction tables

In [10]:
#Load data into Product table
with open('bhejane_covid_essentials_Products.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['Width'],i['Length'],i['Height'], i['Barcode'], i['Quantity'], i['Brand'], i['NavigationPath'], i['Colour'], i['StockCountry'], i['ProductDescription'],i['PackType'],i['Volume_litre'],i['Warranty'],i['Weight_kg'],i["ItemDescription"],i['Price']) for i in dr]

cursor.executemany("INSERT INTO Products VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)
conn.commit()

with open('bhejane_covid_essentials_Transactions.csv','r') as fin: # `with` statement available in 2.5+
    # csv.DictReader uses first line in file for column headings by default
    dr = csv.DictReader(fin) # comma is default delimiter
    to_db = [(i['CartID'],i['Barcode'], i['Total'], i['UserName'], i['InvoiceDate']) for i in dr]

cursor.executemany("INSERT INTO Transactions VALUES (?, ?, ?, ?, ?);", to_db)
conn.commit()

<a id='ERD'></a>
### 5. Denormalized Database Tables

<br>

<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/Denormalized_Tables.png" alt="Denormalized Tables" border="0">

#### 5.1 Exploring the Denormalized Tables
We need to familiarize ourselve with the data given in the two tables by writing our own SQL queries to explore properties of the dataset.i.e Looking for data inconsistencies, anormalies, redundancies etc to guide the normalization process. 

In [11]:
%%read_sql
-- Exploring the dataset
SELECT * FROM Products
LIMIT 10;

Query started at 12:21:05 AM W. Central Africa Standard Time; Query executed in 0.00 m

,Width,Length,Height,Barcode,Quantity,Brand,NavigationPath,Colour,StockCountry,ProductDescription,PackType,Volume_litre,Warranty,Weight_kg,ItemDescription,Price
0,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,1 x Hikvision 1080P Bullet camera,399.0
1,,,,300507946,493.0,Hikvision,Computers & Tablets / Smart Home & Connected L...,,,Hikvision 1080P 2MP Turbo HD IR Bullet Camera,,0.0,Limited (6 months),,Manual,399.0
2,,,,10325354918,467.0,ZEE,Fashion / Accessories / Scarves,Grey,,ZEE 3-in-1 Unisex Gaiter,,0.0,Limited (6 months),,,139.0
3,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Makeup / Face / Found...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
4,,,,27131187035,275.0,Estee Lauder,Beauty / Luxury Beauty / Shop By Brand / Estee...,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
5,,,,27131187035,275.0,Estee Lauder,Beauty / Makeup / Face / Foundation,Fresco,South Africa,Estee Lauder Double Wear Stay In Place Makeup,,0.0,Non-Returnable,,,655.0
6,,,,39800085139,247.0,Energizer,Cameras / Cameras & Lenses / Camera Accessorie...,,South Africa,Energizer 3V CR2450 Lithium Coin Battery,,0.0,Limited (12 months),,,49.0
7,,,,43859499182,386.0,Fellowes,Office & Stationery / Stationery / Office Supp...,,South Africa,Fellowes HFC Free Air Duster - 350ml,,0.0,Limited (6 months),,1 x 350ml Free Air Duster,149.0
8,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0
9,,,,43917915548,338.0,WAHL,Health / Personal Care / Shaving & Hair Remova...,,South Africa,Wahl Home Pro Basic Corded 8 Piece Haircutting...,,0.0,Limited (6 months),,Wahl Home Pro Basic Corded 8 Piece Haircutting...,149.0


### 6. Normalize the given Database tables to the 1st Normal Form (1NF)

Given the below target ERD we will create new tables such the the database conforms to the 1st Normal Form

![1st Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/1stNF.png)


We will first create all the tables before attempting to populate them with data, this will help reduce errors that might creep up due to logical dependencies.

#### 6.1 Create the tables required for the 1st Normal Form

In [12]:
%%read_sql
--#Creating tables required for 1NF
DROP TABLE IF EXISTS "Products_1NF";
DROP TABLE IF EXISTS "Transactions_1NF";

CREATE TABLE "Products_1NF"(
    "Barcode" VARCHAR(150),
    "NavigationPath" VARCHAR(150),
    "ItemDescription" VARCHAR(150),
    "Colour" VARCHAR(150),
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    PRIMARY KEY("Barcode","NavigationPath","ItemDescription")
);

CREATE TABLE "Transactions_1NF" (
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    FOREIGN KEY ("Barcode") REFERENCES "Products_1NF" ("Barcode"),
    PRIMARY KEY("CartID", "Barcode", "UserName")
);

Query started at 12:21:07 AM W. Central Africa Standard Time; Query executed in 0.00 m

#### 6.2 Populate the tables you have create in the above section.

Populate the tables such that the database conforms to the 1st Normal Form

In [13]:
%%read_sql
--#Populating the 1NF tables
DELETE FROM "Products_1NF";
DELETE FROM "Transactions_1NF";

INSERT INTO "Products_1NF" ("Barcode","NavigationPath","ItemDescription","Colour","ProductDescription","Brand",
                            "Price","Quantity","PackType","Warranty","StockCountry","Weight_kg",
                            "Volume_litre","Length","Width","Height")
SELECT DISTINCT
     Barcode
    ,NavigationPath
    ,ItemDescription
    ,Colour
    ,ProductDescription
    ,Brand
    ,Price
    ,Quantity
    ,PackType
    ,Warranty
    ,StockCountry
    ,Weight_kg
    ,Volume_litre
    ,Length
    ,Width
    ,Height 
FROM 
    Products;

INSERT INTO "Transactions_1NF"("CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID
    ,Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions;
    

Query started at 12:21:08 AM W. Central Africa Standard Time; Query executed in 0.01 m

###  7. Converting the database into its 2nd Normal Form (2NF).



#### 7.1 Entity Relationship Diagram

![2nd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/2ndNF.png)

#### 7.2 2NF Requirements
To transition from 1NF to 2NF, we need all columns in all tables to not have a partial dependancy on the PK of the table. This means that any tables which have a composite key e.g. `PRIMARY KEY("Barcode","NavigationPath","ItemDescription")` cannot have columns in the table which are dependant on only `Barcode`, `NavigationPath`, or `ItemDescription`. 

In moving to 2NF we will seperate out each of these as a primary key (for their own respective tables), and any columns dependant on one of these columns but not the others, will go into that table. The only columns remaining in the tables will be fully functionally dependant on the primary key of the table. For example, 

Thereby, we remove the partial dependancy, and will be in 2NF. We will create a unique identifier for the `Transactions_2NF`. 

**Second normal form notes**

<br>$\bullet$ Notice that `Total` is only dependant on `CartID` and not on `<CartID,barcode>`. This is a partial dependancy
<br>$\bullet$ Second normal form required no *partial functional dependancy* on the key. 
<br>$\bullet$ This can be achieved in 2 ways: creating a unique identifier per row (so that there is a simple primary key), or breaking the table up into only the columns which are fully dependant on the composite key. 
<br>$\bullet$ We are going to opt for the easier solution, and create a unique simple primary key on the `Transactions_1NF`. We can create a unique primary key by concatenating `CartID||barcode`
<br>$\bullet$ NB - this approach does make it more challenging to get into 3NF! Which would be a good argument to opt for the latter approach. 

#### 7.3 Correcting Data Anomalies

Removiing all transactions that do not correspond to products that are in the database.

These entries were captured incorrectly, and must be removed.

In [14]:
%%read_sql
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 12:21:08 AM W. Central Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,2


In [15]:
%%read_sql
delete from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 12:21:08 AM W. Central Africa Standard Time; Query executed in 0.00 m

In [16]:
%%read_sql
select count(distinct barcode) from Transactions_1NF where barcode not in (select barcode from products_1NF)

Query started at 12:21:09 AM W. Central Africa Standard Time; Query executed in 0.00 m

,count(distinct barcode)
0,0


#### 7.4 FK Requirements

_Note: sqlite does not always enforce foreign key constraints (and it is even more inconsistent with magic commands to enforce FK constraints!), so it is critical that you understand the purpose of the foreign key constraint and therefore are able to enforce the FK constraint yourself!_

To enforce data integrity, and to ensure that strange things cannot happen - we will have a FK constraint. A foreign key forces all values of the FK, to be limited to only the values which exist as entries in the PK column of the table for which the aforementioned table is a foreign key to. For example, we cannot sell a product (in our Transactions table) which we do not have in our Products table. 


#### 7.5 Client Use-Case Requirements


For all tables which are going to be used, consider the table utility relative to the Products (Products_2NF) table. We will be removing 'useless' entries as we take the data from 1NF to 2NF. For the sake of ease-of-use for the client, we will be removing all rows in the 2NF tables `PackageContents_2NF` and `Colours_2NF` which have an empty value for the attributes that the client will use them for - these are the `ItemDescription` and `Colour` columns, respectively. 

_using a `where` clause, we will remove instances of the value being an empty string, or a `null` value. This will need to be considered for the `ItemDescription` and for the `Colour` when data is inserted into the corresponding tables above (i.e. only inserting data into the `PackageContents_2NF` which has a value for the `ItemDescription`, and only insert data into the `Colours_2NF` table which has a value for `Colour`_

### <font color='blue'> Investigating the anomalies for the tables mentioned </font>

In [17]:
%%read_sql 

SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products
WHERE ItemDescription = '';

Query started at 12:21:09 AM W. Central Africa Standard Time; Query executed in 0.00 m

,ItemDescription,PackType,Warranty
0,,,Limited (6 months)
1,,,Non-Returnable
2,,,Limited (12 months)
3,,,Limited (120 months)
4,,,Limited (180 months)
5,,,Limited (18 months)
6,,,Supplier (12 months)
7,,Single,Limited (6 months)
8,,,Limited (24 months)
9,,,


### Constructing the database above in second normal form (2NF)

In [18]:
%%read_sql

DROP TABLE IF EXISTS "Products_2NF";
DROP TABLE IF EXISTS "Transactions_2NF";
DROP TABLE IF EXISTS "Navigation_2NF";
DROP TABLE IF EXISTS "PackageContents_2NF";
DROP TABLE IF EXISTS "Colours_2NF";

CREATE TABLE Products_2NF(
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Brand" VARCHAR(150),
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    PRIMARY KEY(RegistryID AUTOINCREMENT),
    CONSTRAINT fk_nav FOREIGN KEY('PathID') REFERENCES Navigation_2NF ('PathID'),
    CONSTRAINT fk_pack FOREIGN KEY('ItemID') REFERENCES PackageContents_2NF ('ItemID'),
    CONSTRAINT fk_col FOREIGN KEY('ColourID') REFERENCES Colours_2NF ('ColourID') 
);

CREATE TABLE "Transactions_2NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "UserName" VARCHAR(150) NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName")
    CONSTRAINT fk_trans FOREIGN KEY('Barcode') REFERENCES Products_2NF ('Barcode')
);


CREATE TABLE "Navigation_2NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID" AUTOINCREMENT)
);

CREATE TABLE "Colours_2NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
);

CREATE TABLE "PackageContents_2NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
);


Query started at 12:21:11 AM W. Central Africa Standard Time; Query executed in 0.00 m

### Populating the database above in second normal form (2NF)

Firstly, let's consider the entries which are `null` and `= ''`. And we will only insert the relevant entries. 

In [19]:
%%read_sql
-- #Populating the tables so that they conform to 2NF

DELETE FROM "Products_2NF";
DELETE FROM "Transactions_2NF";
DELETE FROM "Navigation_2NF";
DELETE FROM "PackageContents_2NF";
DELETE FROM "Colours_2NF";


INSERT INTO "Navigation_2NF"("NavigationPath")
SELECT DISTINCT 
    NavigationPath 
FROM
    Products_1NF;
    
INSERT INTO "PackageContents_2NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT 
    ItemDescription,
    PackType,
    Warranty 
FROM
    Products_1NF
WHERE ItemDescription <> ''
        ;
    
INSERT INTO "Colours_2NF"("Colour")
SELECT DISTINCT 
    Colour
FROM
    Products_1NF
WHERE Colour <> ''
        ;

Query started at 12:21:16 AM W. Central Africa Standard Time; Query executed in 0.01 m

A more complicated scenario below as we insert into `Products_2NF`. We will insert from the Products_1NF table. We can use LEFT JOIN, because the data in each of these tables that `Products_1NF` has a FK to, originally came from `Products_1NF`, so we are gauranteed to get matches back. Normally however, what you would get from the FK restriction is the requirement to use _parent-table_ `LEFT JOIN` _child-table_. 

In [20]:
%%read_sql 

INSERT INTO "Products_2NF" ("PathID","ItemID","ColourID","Barcode","ProductDescription","Brand","Price",
                            "Quantity","StockCountry","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    PathID,
    ItemID,
    ColourID,
    Products_1NF.Barcode,
    ProductDescription,
    Brand,
    Price,
    Quantity,
    StockCountry,
    Weight_kg, 
    Volume_litre, 
    Length,
    Width,
    Height 
FROM 
    Products_1NF
LEFT JOIN Navigation_2NF ON Products_1NF.NavigationPath = Navigation_2NF.NavigationPath 
LEFT JOIN PackageContents_2NF ON Products_1NF.ItemDescription = PackageContents_2NF.ItemDescription
LEFT JOIN Colours_2NF ON Products_1NF.Colour = Colours_2NF.Colour

Query started at 12:21:17 AM W. Central Africa Standard Time; Query executed in 0.01 m

Last but not least, we insert into the `Transactions_2NF`table.

In [21]:
%%read_sql 

INSERT INTO "Transactions_2NF"("CartID||Barcode||UserName","CartID","Barcode","UserName","InvoiceDate","Total")
SELECT DISTINCT 
    CartID||Transactions_1NF.Barcode||UserName
    ,CartID
    ,Transactions_1NF.Barcode
    ,UserName
    ,InvoiceDate
    ,Total
FROM
    Transactions_1NF
LEFT JOIN Products_2NF ON Transactions_1NF.Barcode = Products_2NF.Barcode ;


Query started at 12:21:17 AM W. Central Africa Standard Time; Query executed in 0.00 m

<a id='Target_ERD'></a>
### 8. Convert the table into its 3rd Normal Form (3NF)


<br>

![3rd Normal Form](https://raw.githubusercontent.com/Explore-AI/Pictures/master/SQL4DS/SQL_Predict/3rdNF.png)

In [103]:
%%read_sql
SELECT p.Barcode, ct.CartID, ct.Total, u.UserName FROM Products_3NF AS p

INNER JOIN Transactions_3NF as t
ON t.Barcode== p.Barcode

INNER JOIN Users_3NF as u
ON t.UserID == u.UserID

INNER JOIN Locations_3NF AS l
ON p.LocationID == l.LocationID

INNER JOIN Colours_3NF AS c
ON p.ColourID == c.ColourID

INNER JOIN Carts_3NF AS ct
on t.CartID== ct.CartID

WHERE c.Colour== 'Black' AND l.StockCountry == 'South Africa' AND SUBSTR(InvoiceDate, 6,2)% 2!= 0

Query started at 11:55:23 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Barcode,CartID,Total,UserName
0,741311301174,7,1999.0,Cindy
1,5000394020146,18,189.0,Hester
2,6001246636682,32,93.0,Conrad
3,6925281939952,82,799.0,Carel
4,6925281939952,82,799.0,Carel
5,6925281939952,82,799.0,Carel
6,6925281939952,82,799.0,Carel
7,6925281939952,82,799.0,Carel
8,6941428152961,85,798.0,Lacin
9,6941428152961,85,798.0,Lacin


In [60]:
%%read_sql
SELECT COUNT* FROM Transactions_3NF

Query started at 04:15:29 PM W. Central Africa Standard Time; Query executed in 0.00 m

,CartID||Barcode||UserName,CartID,UserID,Barcode
0,143859499182Hendrik,1,1,43859499182
1,2614143543746Faristha,2,2,614143543746
2,3619659141059JOHAN,3,3,619659141059
3,4638142992763Sandra,4,4,638142992763
4,5638142992763Sheila,5,5,638142992763
...,...,...,...,...
270,191MPTAL72849955Eathon,191,157,MPTAL72849955
271,1926002305001472Janet,192,158,6002305001472
272,1929336232000831Janet,192,158,9336232000831
273,192MPTAL57588104Janet,192,158,MPTAL57588104


In [22]:
%%read_sql

DROP TABLE IF EXISTS "Transactions_3NF";
DROP TABLE IF EXISTS "Carts_3NF";
DROP TABLE IF EXISTS "Products_3NF";
DROP TABLE IF EXISTS "Users_3NF";
DROP TABLE IF EXISTS "Navigation_3NF";
DROP TABLE IF EXISTS "PackageContents_3NF";
DROP TABLE IF EXISTS "Colours_3NF";
DROP TABLE IF EXISTS "Brands_3NF";
DROP TABLE IF EXISTS "Locations_3NF";

CREATE TABLE "Products_3NF" (
    "RegistryID" INTEGER NOT NULL,
    "Barcode" VARCHAR(150) NOT NULL,
    "ProductDescription" VARCHAR(150) NOT NULL,
    "Price" REAL NOT NULL,
    "Quantity" INTEGER NOT NULL,
    "Weight_kg" REAL,
    "Volume_litre" REAL,
    "Length" REAL,
    "Width" REAL,
    "Height" REAL,
    "PathID" INTEGER,
    "ItemID" INTEGER,
    "ColourID" INTEGER,
    "BrandID" INTEGER,
    "LocationID" INTEGER,
    PRIMARY KEY("RegistryID" AUTOINCREMENT),
    FOREIGN KEY("PathID") REFERENCES "Navigation_3NF" ("PathID"),
    FOREIGN KEY("ItemID") REFERENCES "PackageContents_3NF" ("ItemID"),
    FOREIGN KEY("ColourID") REFERENCES "Colours_3NF" ("ColourID") ,
    FOREIGN KEY ("BrandID") REFERENCES "Brands_3NF" ("BrandID"),
    FOREIGN KEY ("LocationID") REFERENCES "Locations_3NF" ("LocationID")
);


CREATE TABLE "Carts_3NF" (
    "CartID" INTEGER NOT NULL,
    "InvoiceDate" DATETIME NOT NULL,
    "Total" REAL NOT NULL,
    PRIMARY KEY ("CartID")
);


CREATE TABLE "Users_3NF"(
    "UserID" INTEGER NOT NULL,
    UserName VARCHAR(50) NOT NULL,
    PRIMARY KEY("UserID" AUTOINCREMENT)
);


Query started at 12:21:21 AM W. Central Africa Standard Time; Query executed in 0.00 m

### <font color='blue'> Creating the remaining tables mentioned </font>

In [23]:
%%read_sql

CREATE TABLE "Transactions_3NF" (
    "CartID||Barcode||UserName" VARCHAR(150) NOT NULL,
    "CartID" INTEGER NOT NULL,
    "UserID" INTEGER,
    "Barcode" VARCHAR(150) NOT NULL,
    PRIMARY KEY("CartID||Barcode||UserName"),
    FOREIGN KEY('Barcode') REFERENCES "Products_3NF" ('Barcode'),
    FOREIGN KEY('UserID') REFERENCES "Users_3NF" ('UserID'),
    FOREIGN KEY('CartID') REFERENCES "Carts_3NF" ('CartID')
    );
    
    
CREATE TABLE "Navigation_3NF"(
    "PathID" INTEGER NOT NULL,
    "NavigationPath" VARCHAR(150),
    PRIMARY KEY("PathID")
    );
    
CREATE TABLE "PackageContents_3NF"(
    "ItemID" INTEGER NOT NULL,
    "ItemDescription" VARCHAR(150),
    "PackType" VARCHAR(150),
    "Warranty" VARCHAR(150),
    PRIMARY KEY("ItemID" AUTOINCREMENT)
    );

CREATE TABLE "Colours_3NF"(
    "ColourID" INTEGER NOT NULL,
    "Colour" VARCHAR(150),
    PRIMARY KEY("ColourID" AUTOINCREMENT)
    );

CREATE TABLE "Brands_3NF"(
    "BrandID" INTEGER NOT NULL,
    "Brand" VARCHAR(150),
    PRIMARY KEY("BrandID" AUTOINCREMENT)
    );
    
CREATE TABLE "Locations_3NF"(
    "LocationID" INTEGER NOT NULL,
    "StockCountry" VARCHAR(150),
    PRIMARY KEY("LocationID" AUTOINCREMENT)
    )


Query started at 12:21:22 AM W. Central Africa Standard Time; Query executed in 0.00 m

In [24]:
%%read_sql

--# Populating the tables to that they conform to the 3rd Normal Form
DELETE FROM "Products_3NF";
DELETE FROM "Transactions_3NF";
DELETE FROM "Carts_3NF";
DELETE FROM "Users_3NF";
DELETE FROM "Navigation_3NF";
DELETE FROM "PackageContents_3NF";
DELETE FROM "Colours_3NF";
DELETE FROM "Brands_3NF";
DELETE FROM "Locations_3NF";

INSERT INTO "Users_3NF" ("UserName")
SELECT DISTINCT UserName 
FROM Transactions_2NF;


Query started at 12:21:25 AM W. Central Africa Standard Time; Query executed in 0.00 m

### <font color='blue'> Insert data `Carts_3NF`.   </font>

There are duplicate values coming from `Transactions_2NF` - we retain the `distinct` combinations of `CartID`,`InvoiceDate`,`Total` only.

In [25]:
%%read_sql

INSERT INTO "Carts_3NF"("CartID","InvoiceDate","Total")
SELECT DISTINCT
    CartID
    ,InvoiceDate
    ,Total
FROM Transactions_2NF


Query started at 12:21:25 AM W. Central Africa Standard Time; Query executed in 0.01 m

### <font color='blue'>Insert into the other tables below. </font>

Taking note of duplicate occurences by using `SELECT distinct` to avoid inserting redundant rows. 


In [26]:
%%read_sql

INSERT INTO "Locations_3NF"("StockCountry")
SELECT DISTINCT
    StockCountry
FROM
    Products_2NF
WHERE StockCountry <> '';

INSERT INTO "Brands_3NF"("Brand")
SELECT DISTINCT
    Brand
FROM
    Products_2NF
WHERE Brand <> '';

INSERT INTO "Colours_3NF"("Colour")
SELECT DISTINCT
    Colour
FROM 
    Colours_2NF;

INSERT INTO "PackageContents_3NF"("ItemDescription","PackType","Warranty")
SELECT DISTINCT
    ItemDescription
    ,PackType
    ,Warranty
FROM
    PackageContents_2NF;

INSERT INTO "Navigation_3NF"("NavigationPath")
SELECT DISTINCT 
    NavigationPath
FROM
    Navigation_2NF;

Query started at 12:21:27 AM W. Central Africa Standard Time; Query executed in 0.01 m

### <font color='blue'>Insert into `Products_3NF`. </font>

In [27]:
%%read_sql

INSERT INTO "Products_3NF" ("PathID","ItemID","ColourID","BrandID","LocationID",
                            "Barcode","ProductDescription","Price",
                            "Quantity","Weight_kg","Volume_litre",
                            "Length","Width","Height")
SELECT
    P2.PathID,
    P2.ItemID,
    P2.ColourID,
    B.BrandID,
    L.LocationID,
    P2.Barcode,
    P2.ProductDescription,
    P2.Price,
    P2.Quantity,
    P2.Weight_kg,
    P2.Volume_litre,
    P2.Length,
    P2.Width,
    P2.Height
    
FROM
    Products_2NF AS P2
LEFT JOIN Brands_3NF AS B ON B.Brand = P2.Brand
LEFT JOIN PackageContents_3NF AS PC ON P2.ItemID= PC.ItemID
LEFT JOIN Colours_3NF AS C ON C.ColourID = P2.ColourID
LEFT JOIN Locations_3NF AS L ON L.StockCountry= P2.StockCountry
LEFT JOIN Navigation_3NF AS N ON N.PathID = P2.PathID


    

Query started at 12:21:27 AM W. Central Africa Standard Time; Query executed in 0.02 m

### <font color='blue'>Insert into `Transactions_3NF`.   </font> 

In [28]:
%%read_sql

INSERT INTO "Transactions_3NF"("CartID||Barcode||UserName","CartID","Barcode","UserID")
SELECT DISTINCT
    Tr.CartID||Tr.Barcode||Tr.UserName,
    Tr.CartID,
    Tr.Barcode,
    U3.UserID

FROM Transactions_2NF AS Tr

LEFT JOIN Products_3NF AS P3 ON Tr.Barcode= P3.Barcode
LEFT JOIN Users_3NF AS U3 ON Tr.UserName= U3.UserName
LEFT JOIN Carts_3NF AS C3 ON Tr.CartID= C3.CartID



Query started at 12:21:28 AM W. Central Africa Standard Time; Query executed in 0.01 m

### <font color='blue'>Investigating the DB.   </font>


| Table Name | Count |
| --- | --- |
| Brands_3NF | 232 |
| Carts_3NF |  190 |
| Colours_3NF | 17 |
| Locations_3NF | 2 |
| Navigation_3NF | 396 |
| PackageContents_3NF | 600 |
| Products_3NF | 1214 |
| Transactions_3NF | 275 |
| Users_3NF | 158 |

In [29]:
%%read_sql 

-- confirming the number of entries in the tables with the image displayed above
select 'Products_3NF' as table_name, count(*) from Products_3NF
UNION
select 'Transactions_3NF' as table_name, count(*) from Transactions_3NF
UNION
select 'Users_3NF' as table_name, count(*) from Users_3NF
UNION
select 'Navigation_3NF' as table_name, count(*) from Navigation_3NF
UNION
select 'PackageContents_3NF' as table_name, count(*) from PackageContents_3NF
UNION
select 'Colours_3NF' as table_name, count(*) from Colours_3NF
UNION
select 'Brands_3NF' as table_name, count(*) from Brands_3NF
UNION
select 'Locations_3NF' as table_name, count(*) from Locations_3NF
UNION
select 'Carts_3NF' as table_name, count(*) from Carts_3NF


Query started at 12:21:29 AM W. Central Africa Standard Time; Query executed in 0.00 m

,table_name,count(*)
0,Brands_3NF,232
1,Carts_3NF,190
2,Colours_3NF,17
3,Locations_3NF,2
4,Navigation_3NF,396
5,PackageContents_3NF,600
6,Products_3NF,1214
7,Transactions_3NF,275
8,Users_3NF,158


<a id='MCQ_questions'></a>
## 9. Answering basic analytics questions using the database created

Using the transformed database, we will attempt to answer basic analytics questions with the appropriate querries.

Q1) How many unique products does the company have?

In [30]:
%%read_sql
SELECT COUNT(DISTINCT Barcode) AS 'total unique products' FROM Products_3NF

Query started at 12:21:29 AM W. Central Africa Standard Time; Query executed in 0.00 m

,total unique products
0,608


Total unique products is 608

Q2) How many users bought from Bhejane in April 2020?

In [110]:
%%read_sql
SELECT COUNT(Trn.UserID) AS 'No of Users' FROM Transactions_3NF AS Trn

LEFT JOIN Carts_3NF AS Ct
ON Trn.CartID = Ct.CartID

WHERE SUBSTR(Ct.InvoiceDate, 1,7) = '2020-04'


Query started at 12:21:13 PM W. Central Africa Standard Time; Query executed in 0.00 m

,No of Users
0,57


**Total number of users that boought from Bhejane store in April 2020 is 57**

Q3) How many users bought 3 or more items that cost more than R1000?

In [112]:
%%read_sql

SELECT COUNT(Trn.UserID) AS Users, SUM(P.price) AS Cost FROM Carts_3NF c

LEFT JOIN Transactions_3NF AS Trn
ON c.CartID= Trn.CartID

LEFT JOIN Products_3NF AS P
ON Trn.Barcode= P.Barcode
WHERE P.quantity> 3
AND P.price > 1000

Query started at 12:26:56 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Users,Cost
0,27,73751.0


**Only 27 users bought 3 or more items that cost more than R1000**

Q4) Which user made the largest purchase on a single transaction?

In [122]:
%%read_sql
-- Write your query here:
SELECT u.UserName, c.Total FROM Carts_3NF c

LEFT JOIN Transactions_3NF AS Tr
ON c.CartID = Tr.CartID

LEFT JOIN Users_3NF as u
ON Tr.UserID = u.UserID

--GROUP BY u.UserName
ORDER BY c.Total DESC
LIMIT 5

Query started at 08:16:50 PM W. Central Africa Standard Time; Query executed in 0.00 m

,UserName,Total
0,Mandla,10999.0
1,Brigette,10999.0
2,Andreas Peter,3765.0
3,Mzoxolo,3765.0
4,Zanele,3464.0


**The users that made the largest purchase on a single transaction are Mandla and Brigette**

Q5) How many components does the product: "5m Colour Changing RGB LED Strip Light" (MPTAL57588104) come with?

In [124]:
%%read_sql

SELECT COUNT(Barcode) FROM Products_3NF
WHERE ProductDescription = '5m Colour Changing RGB LED Strip Light'

Query started at 08:50:29 PM W. Central Africa Standard Time; Query executed in 0.00 m

,COUNT(Barcode)
0,8


**there are 8 components of the product '5m Colour Changing RGB LED Strip Light" (MPTAL57588104)** 

Q6) How many brands are available at Bhejane?

In [127]:
%%read_sql

SELECT COUNT(BrandID) AS Brands FROM Brands_3NF

Query started at 09:11:28 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Brands
0,232


**There are 232 brands available at Bhejane**

Q7) What is the price of the "Verimark - Floorwiz 2in1 Mop"?

In [37]:
%%read_sql
SELECT Price FROM Products 
WHERE ProductDescription = 'Verimark - Floorwiz 2in1 Mop'

Query started at 12:21:32 AM W. Central Africa Standard Time; Query executed in 0.00 m

,Price
0,179.0
1,179.0
2,179.0


**The product Verimark - Floorwiz 2in1 Mop' IS 179$**

Q8) Calculate the package volume of the "Russell Hobbs - Slow Cooker" using the given dimensions

In [134]:
%%read_sql
SELECT pk.ItemID, ProductDescription, Length*Height*Width AS volume FROM PackageContents_3NF pk

INNER JOIN Products_3NF pr
ON pk.ItemID== pr.ItemID

WHERE ProductDescription== "Russell Hobbs - Slow Cooker"

Query started at 01:05:12 AM W. Central Africa Standard Time; Query executed in 0.00 m

,ItemID,ProductDescription,volume
0,173,Russell Hobbs - Slow Cooker,26928.0
1,174,Russell Hobbs - Slow Cooker,26928.0
2,175,Russell Hobbs - Slow Cooker,26928.0
3,176,Russell Hobbs - Slow Cooker,26928.0


**The volumen of the Russell Hobbs-Slow Cooker package is 26,928cm3 or 26.93 liters**

Q9) Which user made the most transactions in the Year 2020?

In [141]:
%%read_sql

SELECT u.UserName, COUNT(T.'CartID||Barcode||UserName') AS Transactions, SUBSTR(c.InvoiceDate, 1, 4) AS Year FROM Users_3NF AS u

JOIN Transactions_3NF AS T
ON u.UserID= T.UserID

JOIN Carts_3NF AS c
ON T.CartID= c.CartID

WHERE Year == '2020'
GROUP BY u.UserName
ORDER BY Transactions DESC
LIMIT 5

Query started at 01:16:44 AM W. Central Africa Standard Time; Query executed in 0.00 m

,UserName,Transactions,Year
0,Cornelis,9,2020
1,Erlo,8,2020
2,Zanele,6,2020
3,Heather,6,2020
4,Mandla,5,2020


**The user with the highest number of transactions in 2020 is Cornelis with 9 transaction**

Q10) What is the total number of users that shop at Bhejane?

In [42]:
%%read_sql

SELECT COUNT(UserID) FROM Users_3NF

Query started at 12:21:35 AM W. Central Africa Standard Time; Query executed in 0.00 m

,COUNT(UserID)
0,158


**The total number of users is 158**

Q11) What is the record count for the Colours_3NF Table?

In [43]:
%%read_sql
SELECT COUNT(*) FROM Colours_3NF

Query started at 12:21:36 AM W. Central Africa Standard Time; Query executed in 0.00 m

,COUNT(*)
0,17


**The record count for the colours_3NF table is 17**

Q12) What would the total price be if I had the following items in my cart?
* MPTAL57588104
* 5000394203921
* 6932391917652

In [69]:
%%read_sql

SELECT SUM(p.Price) AS 'Total Price'
FROM Products_3NF AS p

WHERE Barcode= '6932391917652' OR Barcode= 'MPTAL57588104' OR Barcode= '5000394203921'

Query started at 05:58:38 PM W. Central Africa Standard Time; Query executed in 0.00 m

,Total Price
0,3982.0


**The total price would be $3,982**

Q13) What is the barcode of the most sold product?

In [73]:
%%read_sql
SELECT RegistryID, t.Barcode, p.ProductDescription, COUNT(t.'CartID||Barcode||UserName') AS Transactions FROM Products_3NF as p

INNER JOIN Transactions_3NF as t
ON p.Barcode == t.Barcode 

GROUP BY p.ProductDescription
ORDER BY Transactions DESC
LIMIT 5

Query started at 06:01:35 PM W. Central Africa Standard Time; Query executed in 0.00 m

,RegistryID,Barcode,ProductDescription,Transactions
0,479,6003000640348,Nutribullet - Blender Original 600W - 8 Piece,16
1,177,4902505163104,"Pilot G-2 0,5 Fine Nib Gel Retractable Pen - B...",12
2,1006,MPTAL00451356,Berlinger Haus 6 Piece Marble Coated Knife Set,12
3,527,6005427110459,Verimark - Twista Pull Chopper,10
4,168,4549292118766,Canon PIXMA TR4540 A4 4-in-1 Wi-Fi Inkjet Printer,10


**The barcode of the most sold product is '6003000640348' with 16 transactions in total**

Q14) What are the products of Cornelis’ cart on the 2020-06-28 0:00:00?

In [78]:
%%read_sql
SELECT p.ProductDescription, u.UserName, c.InvoiceDate from Products_3NF as p

INNER JOIN Transactions_3NF as t
ON p.Barcode== t.Barcode

INNER JOIN Carts_3NF as c
ON t.CartID == c.CartID

INNER JOIN Users_3NF as u
ON t.UserID == u.UserID

WHERE u.UserName== 'Cornelis' AND c.InvoiceDate== '2020-06-28 0:00:00'

GROUP BY ProductDescription

Query started at 06:35:19 PM W. Central Africa Standard Time; Query executed in 0.00 m

,ProductDescription,UserName,InvoiceDate
0,Dettol Body Wash - Shower Gel - Original - 600ml,Cornelis,2020-06-28 0:00:00
1,Protective Face Shield,Cornelis,2020-06-28 0:00:00
2,Sparq Active Resistance Loop Band - Set of 5 w...,Cornelis,2020-06-28 0:00:00
3,Vitality Aloe Vera Gel 100ml,Cornelis,2020-06-28 0:00:00


**The products in Cornelis cart are Dettol Body Wash - Shower Gel - Original - 600ml, Protective Face Shield, Sparq Active Resistance Loop Band - Set of 5 anc Vitality Aloe Vera Gel 100ml.**

Q15) Which users bought locally produced, black-coloured products on odd-numbered months of the year, and what was the total cost (rounded to the nearest integer) of the carts containing these products?

In [142]:
%%read_sql
SELECT p.Barcode, ct.CartID, ROUND(SUM(ct.Total)) AS Cost, u.UserName FROM Products_3NF AS p

INNER JOIN Transactions_3NF as t
ON t.Barcode== p.Barcode

INNER JOIN Users_3NF as u
ON t.UserID == u.UserID

INNER JOIN Locations_3NF AS l
ON p.LocationID == l.LocationID

INNER JOIN Colours_3NF AS c
ON p.ColourID == c.ColourID

INNER JOIN Carts_3NF AS ct
on t.CartID== ct.CartID

WHERE c.Colour== 'Black' AND l.StockCountry == 'South Africa' AND SUBSTR(InvoiceDate, 6,2)% 2!= 0
GROUP BY ct.CartID


Query started at 01:23:59 AM W. Central Africa Standard Time; Query executed in 0.01 m

,Barcode,CartID,Cost,UserName
0,741311301174,7,1999.0,Cindy
1,5000394020146,18,189.0,Hester
2,6001246636682,32,93.0,Conrad
3,6925281939952,82,3995.0,Carel
4,6941428152961,85,2394.0,Lacin
5,7290012291685,87,358.0,Jeandre
6,MPTAL00215561,102,1523.0,DIMPHO


**The users that bought locally produced, black-coloured products on odd-numbered months of the year are Cindy, Hester, Conrad, Carel, Lacin, Jeandre and DIMPHO with a corresponding cost of $1999, $188, $93, $3995, $2394, $358, and $1523 respectively**